In [1]:
import pandas as pd
import numpy as np
from collections import *
from typing import *
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Custom Code imports
import sys
sys.path.insert(0, '/Users/briankalinowski/PycharmProjects/FakeNewsChallenge/Code')
import DataProcessing as pro

In [2]:
fake_news = pd.read_csv('/Users/briankalinowski/Desktop/Data/Kaggle/fake_news.csv')
fake_news = fake_news[fake_news.language == 'english']
fake_news.drop(columns=['uuid', 'ord_in_thread', 'published'], inplace=True)
fake_news = fake_news.dropna(subset=['text'])
fake_news.head()

,author,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,Barracuda Brigade,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,reasoning with facts,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,Barracuda Brigade,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,Fed Up,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,Fed Up,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [3]:
fake_news_content = fake_news[['title', 'text', 'type']]
fake_news_content = fake_news_content.reset_index(drop=True)
fake_news_content = pro.clean_text_column(fake_news_content, 'title')
fake_news_content = pro.clean_text_column(fake_news_content, 'text')
fake_news_content.head()

,title,text,type
0,Muslims BUSTED They Stole Millions In Govt Ben...,Print They should pay all the back all the mon...,bias
1,Re Why Did Attorney General Loretta Lynch Plea...,Why Did Attorney General Loretta Lynch Plead T...,bias
2,BREAKING Weiner Cooperating With FBI On Hillar...,Red State Fox News Sunday reported this morni...,bias
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,bias
4,FANTASTIC! TRUMPS 7 POINT PLAN To Reform Healt...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,bias


In [4]:
# Each article body series as a list of text|
fake_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'fake']['text'])]
bias_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'bias']['text'])]
consiracy_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'conspiracy']['text'])]
hate_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'hate']['text'])]
state_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'state']['text'])]
junksci_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'junksci']['text'])]
satire_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'satire']['text'])]
bs_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'bs']['text'])]

In [5]:
news_tokens = pro.run_ner_noun_parse(bias_content)
# pro.compute_doc_frequency_matrix(news_tokens)

In [12]:
vectorizer = CountVectorizer(min_df=10)
fake_vec = vectorizer.fit_transform(news_tokens)

In [13]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

lda = LatentDirichletAllocation(n_components=10, max_iter=10, learning_method='online', verbose=True)

fake_topics = lda.fit_transform(fake_vec)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [14]:
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print(' '.join([vectorizer.get_feature_names()[i] for i in topic.argsort()[:-top_n - 1:-1]]))
        print('')

In [15]:
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
obama trump john americans cnn barack david donald obamas clinton

Topic 1:
trump donald hillary clinton republican america florida gop new clintons

Topic 2:
new york kelly news times fox gingrich shapiro megyn america

Topic 3:
north carolina national texas bush george cnn state committee democratic

Topic 4:
democrats court republicans supreme trump house donald senate states united

Topic 5:
fbi clinton comey hillary republicans clintons james comeys russia congress

Topic 6:
david joe wnd jones iowa democratic america university facebook committee

Topic 7:
clinton hillary clintons bill house wikileaks wnd hillarys democratic committee

Topic 8:
department justice scott eric act washington wnd doj james american

Topic 9:
jesus states united americans war college christians california america bill



In [16]:
fake_topics

array([[0.1       , 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.1875144 , 0.01666819, 0.01666695, ..., 0.01666705, 0.01666957,
        0.01666714],
       [0.01250042, 0.01250126, 0.26576844, ..., 0.01250361, 0.01250328,
        0.0125    ],
       ...,
       [0.0125013 , 0.19725162, 0.51234205, ..., 0.20290131, 0.01250032,
        0.0125    ],
       [0.03333485, 0.03333368, 0.03333477, ..., 0.03333403, 0.03333333,
        0.03333333],
       [0.03333444, 0.03333493, 0.37661419, ..., 0.0333342 , 0.03333413,
        0.03333333]])